In [2]:
import torch
import scvi
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()
assert(scvi.__version__=='0.16.3')

from gradients import run_expected_jacobian_scanvi, run_integrated_jacobian_scanvi

Global seed set to 0
2022-09-21 16:30:54.226666: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.5 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.10.1 pynndescent==0.5.7


In [3]:
base_path = '/home/icb/yuge.ji/projects/feature-attribution-sc'  # should be changed to shared dir when I can find one

In [6]:
hlca_path = '/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data/HLCA_core_h5ads/HLCA_v1_integration/HLCA_v1_scANVI_input.h5ad'
adata = sc.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 587218 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

In [7]:
model = scvi.model.SCANVI.load('/home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/scanvi_model/', adata)
model

INFO     File /home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/sca
         nvi_model/model.pt already downloaded                                               


/home/icb/sergei.rybakov/miniconda3/envs/feature/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/home/icb/sergei.rybakov/miniconda3/envs/feature/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(


ScanVI Model with the following params: 
unlabeled_category: unlabeled, n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, 
dispersion: gene, gene_likelihood: nb
Training status: Trained

In [9]:
#scdl = model._make_data_loader(adata=adata, shuffle=True, batch_size=100)

In [10]:
#inpt_dict = batch_to_dict_scanvi(next(scdl.__iter__()))
#inpt_dict["x"].requires_grad = True

In [11]:
#expected_jacobian(model.module.classify, inpt_dict, "x", prime_inpt)

In [12]:
adata.obs["dataset"].value_counts()

Banovich_Kropski_2020        122319
Barbry_Leroy_2020             74728
Nawijn_2021                   70591
Misharin_2021                 65326
Krasnow_2020                  60990
Misharin_Budinger_2018        41265
Meyer_2019                    35674
Jain_Misharin_2021_10Xv2      33144
Seibold_2020_10Xv3            21466
Lafyatis_Rojas_2019_10Xv2     21271
Teichmann_Meyer_2019          12971
Jain_Misharin_2021_10Xv1      12422
Seibold_2020_10Xv2            12127
Lafyatis_Rojas_2019_10Xv1      2924
Name: dataset, dtype: int64

In [13]:
indices = np.where(adata.obs["dataset"] == "Meyer_2019")[0]

In [14]:
dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=1000)
dl_prime = model._make_data_loader(adata=adata, shuffle=True, indices=indices, batch_size=1000)

In [24]:
avg_abs_exp_jac = None
avg_exp_jac = None

In [25]:
# sum abs values
for ds in adata.obs["dataset"].unique():
    print(ds)
    
    indices = np.where(adata.obs["dataset"] == ds)[0]
    batch_size = min(2000, int(len(indices) / 11))
    print(batch_size)
    
    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=batch_size)
    dl_prime = model._make_data_loader(adata=adata, shuffle=True, indices=indices, batch_size=batch_size)
    
    exp_jac_ds = run_expected_jacobian_scanvi(model.module.classify, dl_base, dl_prime, apply_abs=True, sum_obs=True)
    if avg_abs_exp_jac is None:
        avg_abs_exp_jac = exp_jac_ds
    else:
        avg_abs_exp_jac += exp_jac_ds
        
# just sum
for ds in adata.obs["dataset"].unique():
    print(ds)
    
    indices = np.where(adata.obs["dataset"] == ds)[0]
    batch_size = min(2000, int(len(indices) / 11))
    print(batch_size)
    
    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=batch_size)
    dl_prime = model._make_data_loader(adata=adata, shuffle=True, indices=indices, batch_size=batch_size)
    
    exp_jac_ds = run_expected_jacobian_scanvi(model.module.classify, dl_base, dl_prime, apply_abs=False, sum_obs=True)
    if avg_exp_jac is None:
        avg_exp_jac = exp_jac_ds
    else:
        avg_exp_jac += exp_jac_ds

Teichmann_Meyer_2019
1179
Misharin_Budinger_2018
2000
Krasnow_2020
2000
Seibold_2020_10Xv3
1951
Banovich_Kropski_2020
2000
Nawijn_2021
2000
Misharin_2021
2000
Seibold_2020_10Xv2
1102
Barbry_Leroy_2020
2000
Lafyatis_Rojas_2019_10Xv1
265
Lafyatis_Rojas_2019_10Xv2
1933
Meyer_2019
2000
Jain_Misharin_2021_10Xv2
2000
Jain_Misharin_2021_10Xv1
1129
Teichmann_Meyer_2019
1179
Misharin_Budinger_2018
2000
Krasnow_2020
2000
Seibold_2020_10Xv3
1951
Banovich_Kropski_2020
2000
Nawijn_2021
2000
Misharin_2021
2000
Seibold_2020_10Xv2
1102
Barbry_Leroy_2020
2000
Lafyatis_Rojas_2019_10Xv1
265
Lafyatis_Rojas_2019_10Xv2
1933
Meyer_2019
2000
Jain_Misharin_2021_10Xv2
2000
Jain_Misharin_2021_10Xv1
1129


In [34]:
avg_abs_integr_jac = None
avg_integr_jac = None

In [35]:
# sum abs values
for ds in adata.obs["dataset"].unique():
    print(ds)
    
    indices = np.where(adata.obs["dataset"] == ds)[0]
    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=2000)
    
    integr_jac_ds = run_integrated_jacobian_scanvi(model.module.classify, dl_base, apply_abs=True, sum_obs=True)
    if avg_abs_integr_jac is None:
        avg_abs_integr_jac = integr_jac_ds
    else:
        avg_abs_integr_jac += integr_jac_ds
        
# just sum
for ds in adata.obs["dataset"].unique():
    print(ds)
    
    indices = np.where(adata.obs["dataset"] == ds)[0]
    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=2000)
    
    integr_jac_ds = run_integrated_jacobian_scanvi(model.module.classify, dl_base, apply_abs=False, sum_obs=True)
    if avg_integr_jac is None:
        avg_integr_jac = integr_jac_ds
    else:
        avg_integr_jac += integr_jac_ds

Teichmann_Meyer_2019
Misharin_Budinger_2018
Krasnow_2020
Seibold_2020_10Xv3
Banovich_Kropski_2020
Nawijn_2021
Misharin_2021
Seibold_2020_10Xv2
Barbry_Leroy_2020
Lafyatis_Rojas_2019_10Xv1
Lafyatis_Rojas_2019_10Xv2
Meyer_2019
Jain_Misharin_2021_10Xv2
Jain_Misharin_2021_10Xv1
Teichmann_Meyer_2019
Misharin_Budinger_2018
Krasnow_2020
Seibold_2020_10Xv3
Banovich_Kropski_2020
Nawijn_2021
Misharin_2021
Seibold_2020_10Xv2
Barbry_Leroy_2020
Lafyatis_Rojas_2019_10Xv1
Lafyatis_Rojas_2019_10Xv2
Meyer_2019
Jain_Misharin_2021_10Xv2
Jain_Misharin_2021_10Xv1


In [47]:
from scvi import REGISTRY_KEYS
labels = model.adata_manager.get_state_registry(REGISTRY_KEYS.LABELS_KEY)['categorical_mapping']

In [49]:
pd.DataFrame(avg_abs_exp_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("absolute_sum_expected_grads.csv", index_label="Gene")

In [50]:
pd.DataFrame(avg_exp_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("sum_expected_grads.csv", index_label="Gene")

In [51]:
pd.DataFrame(avg_abs_integr_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("absolute_sum_integrated_grads.csv", index_label="Gene")

In [52]:
pd.DataFrame(avg_integr_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("sum_integrated_grads.csv", index_label="Gene")